Tratamento do Dataset

In [ ]:
# Bibliotecas 
import pandas as pd
import datetime as dt
import re
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', None)
data=pd.read_csv('train.csv') # dataset do projeto

#### data INFO

In [ ]:
# data.dtypes
data.info() #milage
# data.head()
# data.tail()
# data.shape #(3207, 12)
# data.nunique() # valores unicos para cada coluna

In [ ]:
# modelo mais antigo e mais novo:
antigo=data['model_year'].min()
novo=data['model_year'].max()
print(antigo,novo)

#### correção: 'milage' -> INT

In [ ]:
#milage:
valores_corrigidos=[]
for milage in data['milage']: # para cada quilometragem do dataset
    separa=re.split(r'[ ]',milage) # sepagar o numero do mi
    numero=separa[0] # numero
    letra=separa[1] # mi
    verifica=re.search(r'^[0-9]+[,]?[0-9]+$',numero) # verifica se o numeros sao sempre iguais(com casas decimais ou não)
    verifica2=re.search(r'^mi\.$',letra) # verifica se a letra é sempre mi
    if verifica and verifica2: # se seguir o padra numero + mi:
        numero=int(re.sub(r',','',numero)) # retira a , dos numeros e passa para inteiro ( estavam em obj)
        valores_corrigidos.append(numero)
    else: 
        print(milage,False) # tem dados diferentes no dataset

data['milage']=valores_corrigidos

In [ ]:
#verificaçao
for m in data['milage']:
    if not isinstance(m, (int)):
        print('dado incorreto')
# tudo certo

In [ ]:
data.describe()

#### NULL's

In [ ]:
data.isnull().sum()

#### counts

In [ ]:
data['fuel_type'].value_counts() # –  38 ( existem 38 linhas com (-) -> nao se sabe)
data['accident'].value_counts()
data['clean_title'].value_counts() # Yes -> 2740, os restantes são valores nulos (nan)
# data['clean_title'].unique()
# verificar outros tipos de dizer valores nulos: ( como (-) por exemplo)
data['brand'].value_counts() # tudo certo
data['model'].value_counts() # tudo certo
data['model_year'].value_counts() # tudo certo
data['engine'].value_counts() # – 38 -> nao se sabe
data['transmission'].value_counts() # 4 -> nao se sabe 
data['ext_col'].value_counts()# 11 -> nao se sabe
data['int_col'].value_counts() # 98 -> nao se sabe
for preco in data['price']: # tudo certo
    if not isinstance(preco, int):
        print('erro')

In [ ]:
len(data['milage'])
# len(data['milage'].unique())

##### --> nº de velocidades na transmissão existentes no dataset

In [ ]:
d2=list(data['transmission'].unique())
alls = [int(numero) for string in d2 for numero in re.findall(r'\d+', string)]
list(set(alls))

#### verificar significado de '-' e NaN para o tipo de combustível

In [ ]:
len(data.loc[data['fuel_type'] == '–'])

In [ ]:
data.loc[(data['fuel_type'] == '–') & (data['engine'] != '–')] 

###### OS CARROS PARA OS QUAIS O FUEL_TYPE É '-' NÃO INDICADO, TAMBÉM NÃO SE CONHECE A CONFIGURAÇÃO DO MOTOR

In [ ]:
# VERIFICAR QUE OS CARROS COM NAN NO TIPO DE COMBUSTÍVEL SÃO CARROS ELÉTRICOS
elec=data[data['fuel_type'].isnull()]
pattern = re.compile(r'\bElectric\b', flags=re.IGNORECASE)
contains_electric = elec['engine'].str.contains(pattern, na=False)
# Selecionar todas as linhas que não contêm 'Electric' na coluna 'engine'
elec[~contains_electric]
# elec[contains_electric].head(10)

###### os carros com NaN para 'fuel_type' são carros elétricos
Tesla --> elétrico
Standard Range Battery --> elétrico
111.2Ah / FR 70kW / RR 160kW (697V) --> especificidade de baterias

### gráficos

#### BOXPLOT

In [ ]:
data.plot(kind='box',figsize=(15,6),subplots=True) # grafico do codigo acima

In [ ]:
data[data['model_year']<1990] # outlier do 1º gráfico

In [ ]:
data[data['milage']>350000] # outlier do 2º gráfico

In [ ]:
data[data['price']>1500000] # outlier 3º gráfico

#### BARPLOT

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Supondo que 'data' seja seu DataFrame com colunas de atributos e 'preco'
# Vamos calcular a média do preço para cada atributo (exceto a última coluna)
data['fuel_type'].fillna('Elétrico', inplace=True)

# Lista para armazenar os gráficos gerados
colunas = ['brand', 'model', 'model_year', 'fuel_type', 'engine', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title']
num_linhas = 3
num_colunas = 3

fig, axs = plt.subplots(num_linhas, num_colunas, figsize=(15, 10))

# Iterar sobre as colunas do DataFrame
for i, column in enumerate(colunas[:-1]):
    # Calcular a média do preço para cada valor único na coluna
    med = data.groupby(column)['price'].mean()
    top = med.sort_values(ascending=False).head(10)
    
    # Truncate long labels and append ellipsis
    truncated_labels = [str(val)[:10] + '...' if len(str(val)) > 10 else str(val) for val in top.index]

    # Determine the subplot index
    linha = i // num_colunas
    coluna = i % num_colunas

    # Plotar o gráfico de barras para a média do preço por valor
    axs[linha, coluna].bar(truncated_labels, top.values, color='deepskyblue')
    axs[linha, coluna].set_title(f'Média de Preço por {column.upper()}')
    axs[linha, coluna].set_ylabel('Média de Preço')
    axs[linha, coluna].tick_params(axis='x', rotation=45)  # Rotacionar rótulos do eixo x

# Ajustar o layout para evitar sobreposição
plt.tight_layout(rect=[0, 0.1, 1, 2])
plt.show()


#### análise dos modelos com preço mais altos

In [ ]:
d=data.sort_values(by='price', ascending=False).head(10)
d[['brand', 'model', 'price']].head(10)

#### LINEPLOT

In [ ]:
d = data.groupby('milage')['price'].mean()
d=d.sort_index()
plt.scatter(d.index, d.values, marker='o', linestyle='-')
plt.title('Preço Médio em Função do Número de Quilômetros')
plt.xlabel('Quilometragem')
plt.ylabel('Preço Médio')
plt.xticks(rotation=45)

plt.tight_layout()  # Ajuste automático da disposição para evitar sobreposição
plt.show()


#### o preço diminui a medida que o nº de quilometros aumenta

In [ ]:
summary_stats = data.groupby(['brand', 'model_year'])['price'].describe()
print(summary_stats)

# [ENGINE] novos atributos 

In [ ]:
data['Potencia'] = data['engine'].str.extract(r'(\d+\.\d+)HP?')
data['Capacidade_Motor'] = data['engine'].str.extract(r'(\d+\.\d+|\d+)\s*(?:L|Liter)')
data['Numero_Cilindros'] = data['engine'].str.extract(r'(?:V(\d+)|I-(\d+)|I(\d+)|(\d+) Cylinder)').apply(lambda x: next(filter(lambda y: pd.notna(y), x), None), axis=1)
# data['Tipo_Combustivel'] = data['engine'].str.extract(r'(Gasoline Fuel|Flexible Fuel|Electric)')
data['Numero_Valvulas'] = data['engine'].str.extract(r'( \d+)V')

data['Potencia'] = pd.to_numeric(data['Potencia'], errors='coerce')
data['Capacidade_Motor'] = pd.to_numeric(data['Capacidade_Motor'], errors='coerce')
data['Numero_Cilindros'] = pd.to_numeric(data['Numero_Cilindros'], errors='coerce')
data['Numero_Valvulas'] = pd.to_numeric(data['Numero_Valvulas'], errors='coerce')

In [ ]:
data.info()

In [ ]:
# data[['engine', 'Potencia', 'Capacidade_Motor',  'Numero_Cilindros',  'Numero_Valvulas' ]].head(50)

In [ ]:
d=len(data[data['Numero_Valvulas'].isnull()])
d
# d[['engine', 'Potencia', 'Capacidade_Motor',  'Numero_Cilindros',  'Numero_Valvulas' ]].head(50)
# data[['engine', 'Potencia', 'Capacidade_Motor',  'Numero_Cilindros',  'Numero_Valvulas' ]].head(10)

#### BARPLOT

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Assuming 'data' is your DataFrame with columns of attributes and 'price'
# Let's calculate the mean price for each attribute (except the last column)

# List to store the generated plots
columns = ['Potencia', 'Capacidade_Motor', 'Numero_Cilindros', 'Numero_Valvulas']

num_linhas = 2
num_colunas = 2

fig, axs = plt.subplots(num_linhas, num_colunas, figsize=(10, 5))

# Iterar sobre as colunas do DataFrame
for i, column in enumerate(columns):
    # Calcular a média do preço para cada valor único na coluna
    med = data.groupby(column)['price'].mean()
    top = med.sort_values(ascending=False).head(13)
    
    # Truncate long labels and append ellipsis
    truncated_labels = [str(val)[:10] + '...' if len(str(val)) > 10 else str(val) for val in top.index]

    # Determine the subplot index
    linha = i // num_colunas
    coluna = i % num_colunas

    # Plotar o gráfico de barras para a média do preço por valor
    axs[linha, coluna].bar(truncated_labels, top.values, color='blueviolet')
    axs[linha, coluna].set_title(f'Média de Preço por {column.upper()}')
    axs[linha, coluna].set_ylabel('Média de Preço')
    axs[linha, coluna].tick_params(axis='x', rotation=45)  # Rotacionar rótulos do eixo x

# Ajustar o layout para evitar sobreposição
plt.tight_layout(rect=[0, 0.1, 1, 2])
plt.show()

In [ ]:
# data.loc[data['Numero_Valvulas'] == 32]

In [ ]:
len(data['Numero_Valvulas'].unique())
sorted(list(data['Numero_Valvulas'].unique()))

In [ ]:
len(data['Numero_Cilindros'].unique())
list(sorted(data['Numero_Cilindros'].unique()))

In [ ]:
# len(data['Capacidade_Motor'].unique())
sorted(list(data['Capacidade_Motor'].unique()))

In [ ]:
sorted(data['Potencia'].unique())

###### verificação de possíveis valores únicos reportados pela análise do boxplot preço / marca no R

In [ ]:
data.loc[data['brand'] == 'Maybach'].head()

In [ ]:
data.to_csv('train_ccols.csv', index=False)

# DATASET W/ ENGINE caract-

In [ ]:
import pandas as pd
import datetime as dt
import re
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', None)
p=pd.read_csv('train_ccols.csv') 

In [ ]:
p.columns

In [ ]:
(p['fuel_type'].unique())

In [ ]:
print(p.duplicated().sum())

# SUBSET 1 características do carro

preço ~ engine atbs + transmission atbs (T2) + fuel_type + model_year (price e milage mesma faixa de valores; novos atributos mesma faixa de valores; ano) -- NORMALIZAR ?

### select columns needed

In [ ]:
data = p.loc[:, ['model_year', 
'fuel_type', 
'transmission','price', 'Potencia', 'Capacidade_Motor', 'Numero_Cilindros', 'Numero_Valvulas']]

In [ ]:
data.describe()

In [ ]:
data.head()

### NULL's substitution

#### ENGINE clone

In [ ]:
m = data['Numero_Valvulas'].mean()
m2 = data['Numero_Cilindros'].mean()
m3 = data['Potencia'].mean()
m4 = data['Capacidade_Motor'].mean()

data['Numero_Valvulas'].fillna(m, inplace=True)
data['Numero_Cilindros'].fillna(m2, inplace=True)
data['Potencia'].fillna(m3, inplace=True)
data['Capacidade_Motor'].fillna(m4, inplace=True)

#### TRANSMISSION SEP 

In [ ]:
# import numpy as np
data['T2'] = data['transmission'].str.extract(r'(\d+)')
data['T2'] = pd.to_numeric(data['T2'], errors='coerce')
m5 = data['T2'].mean()
data['T2'].fillna(round(m5), inplace=True) # arredondar para cima
# 1478 nulls

#### FUEL TYPE

In [ ]:
data['fuel_type'].unique()
data['fuel_type'].isnull().sum()
data['fuel_type'].fillna('Eletric', inplace=True)
mod = data['fuel_type'].mode()[0]
data['fuel_type'] = data['fuel_type'].replace('–', mod)
data['fuel_type'].unique()

In [ ]:
data.head()

#### usar ONE-HOT-ENCODING para FUEL_TYPE

In [ ]:
df_encoded = pd.get_dummies(data['fuel_type'], prefix='fuel_type')

# Concatenando os dataframes original e codificado
df = pd.concat([data, df_encoded], axis=1)

# Removendo a coluna original 'Fuel_Type'
df.drop(columns=['fuel_type'], inplace=True)
df

In [ ]:
len(data['T2'].unique()) # 9
# len(data['fuel_type'].unique()) # 7
# len(data['Numero_Cilindros'].unique()) # 8
# len(data['Potencia'].unique()) # 324
# len(data['Numero_Valvulas'].unique()) # 9
# len(data['Capacidade_Motor'].unique()) # 61

### NORMALIZAÇÃO

In [ ]:
data = data.drop(['transmission'], axis=1)  # Features

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Define the scaler
scaler = MinMaxScaler()

# Fit the scaler on the data and transform the features
data[['price', 'model_year', 'Potencia', 'Capacidade_Motor', 'Numero_Cilindros', 'Numero_Valvulas', 'T2']] = scaler.fit_transform(data[['price', 'model_year', 'Potencia', 'Capacidade_Motor', 'Numero_Cilindros', 'Numero_Valvulas', 'T2']])


In [ ]:
import numpy as np

# Apply logarithmic transformation to skewed features
skewed_features = ['price', 'model_year', 'Potencia', 'Capacidade_Motor', 'Numero_Cilindros', 'Numero_Valvulas', 'T2']  # Specify the skewed features here
data[skewed_features] = np.log1p(data[skewed_features])


### LOF p/ outliers

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import LocalOutlierFactor

nums = data.select_dtypes(include=[np.number])


In [ ]:
lof = LocalOutlierFactor(n_neighbors=20, novelty=False, contamination=0.05)
y_pred = lof.fit_predict(nums)
data['outlier'] = y_pred

In [ ]:
out = data[data['outlier']==-1]
# len(out) # 161 outliers
out.describe()

outlier_indices = data[data['outlier'] == -1].index
o=(p.loc[outlier_indices]).head()
o.describe() # dados outliers

In [ ]:
(p.drop(outlier_indices)).describe() # dados sem os outliers

##### DROPPING OUTLIERS

In [ ]:
criteria = data['outlier'] == -1
clean_data = data[~criteria] # datarfame sem outliers

In [ ]:
clean_data.columns
ck = clean_data.drop(['outlier'], axis=1) # nova variável para fazer o modelo sem os outliers

##### BOXPLOT

In [ ]:
woo = data.drop(['outlier'], axis=1) # nova variável para fazer o modelo sem os outliers
import numpy as np
import matplotlib.pyplot as plt

# Selecionar apenas as colunas numéricas
colunas_numericas = woo.select_dtypes(include=np.number)

# Definir o número de colunas e linhas para os subplots
num_linhas = 2
num_colunas = 4
fig, axs = plt.subplots(num_linhas, num_colunas, figsize=(16, 10))

# Iterar sobre os atributos numéricos e plotar os boxplots em subplots
for i, coluna in enumerate(colunas_numericas.columns):
    linha = i // num_colunas
    coluna_atual = i % num_colunas
    axs[linha, coluna_atual].boxplot(colunas_numericas[coluna])
    axs[linha, coluna_atual].set_title(f'{coluna}')
    axs[linha, coluna_atual].set_xlabel('Coluna')
    axs[linha, coluna_atual].set_ylabel('Valores')
    axs[linha, coluna_atual].grid(True)

# Ajustar o layout dos subplots
plt.tight_layout()

# Mostrar o gráfico
plt.show()


### Separar Data / Treinar e Avaliar Algoritmos

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn import tree
from sklearn.model_selection import train_test_split as split
# from sklearn import datasets
# from slickml.metrics import (
#     RegressionMetrics,
# )  # downloaded from https://github.com/slickml/slick-ml # btw pip install slickml
from matplotlib import pyplot as plt
# import shap

In [ ]:
# X = data.drop(['price'], axis=1)  # Features
# y = data['price'] 

X = ck.drop(['price'], axis=1)  # sem os outliers 
y = ck['price'] 

In [ ]:
# X = pd.get_dummies(X, drop_first=True) # para var. categóricas
# Option 1: Fill missing values, for example with the median or mean
# X.fillna(X.median(), inplace=True)

# Option 2: Drop rows with missing values
# X.dropna(inplace=True)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


#### REGRESSÃO LINEAR

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
model = LinearRegression()
model.fit(X_train, y_train)


In [ ]:
# Make predictions
y_pred = model.predict(X_test)


In [ ]:
# Calculate evaluation metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("R-squared:", r2)


In [ ]:
import matplotlib.pyplot as plt

# Plotting actual vs predicted prices
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, color='blue', alpha=0.5)
plt.title('Actual vs Predicted Car Prices (Multiple Linear Regression)')
plt.xlabel('Actual Price')
plt.ylabel('Predicted Price')
plt.grid(True)
plt.show()


#### RANDOM FOREST

In [ ]:
# train and predict with Random Forest Regressor
RF = RandomForestRegressor()
RF.fit(X_train, y_train)
pred = RF.predict(X_test)

In [ ]:
# Regression metrics for Random Forest
from slickml.metrics import RegressionMetrics

reg_metrics = RegressionMetrics(y_test, pred)
reg_metrics.plot()

# CORRELATION

In [ ]:
import seaborn as sns

dados_numericos = data.select_dtypes(include=['number'])
correlation_matrix = dados_numericos.corr(method='spearman')
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
# preço está bastante correlacionado com potência e ano de fábrico

correlação p/ preço/nº valvulas diminui muito quando se substitui NaN (0.51 para 0.11)

# SUBSET 2

preço ~ milage + clean_title + accident

#### ACCIDENT

In [ ]:
ac_m = data['accident'].mode()[0]
data['accident'].fillna(ac_m, inplace=True)
data['accident'].unique()

#### CLEAN TITLE

In [ ]:
data['clean_title'].fillna('No', inplace=True)
data['clean_title'].unique()

#### CORES

In [ ]:
def simplify_color(color):
    return color.split()[0]

# Aplicar a função à coluna 'Cor'
p['simp_int_col'] = p['int_col'].apply(simplify_color)
p['simp_ext_col'] = p['ext_col'].apply(simplify_color) # de 265 p/ 191
# n vale a pena

# SUBSET 3 (juntar a marca ao subset 1)

In [ ]:
preco_medio_por_marca = p.groupby('brand')['price'].mean()

In [ ]:
def categorizar_preco(preco):
    if preco <= 20000:
        return 'Econômico'
    elif preco <= 50000:
        return 'Intermediário'
    elif preco <= 100000:
        return 'Luxo'
    else:
        return 'Super Luxo'
preco_medio_por_marca = preco_medio_por_marca.map(categorizar_preco)


In [ ]:
preco_medio_por_marca

In [ ]:
# Convertendo para DataFrame
d = preco_medio_por_marca.reset_index()
d.columns = ['brand', 'Categoria']

# Merge com o DataFrame original
p = p.merge(d, on='brand', how='left')


In [ ]:
p.head()